In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing import image

In [2]:
model = load_model('sample/model_13.h5')

In [3]:
# Data preparation
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2, rotation_range=20)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    'dataset/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
# Obtain class indices
class_indices = train_generator.class_indices
# Create a mapping from index to class name
index_to_class = {int(v): k for k, v in class_indices.items()}

Found 15750 images belonging to 45 classes.
Found 3375 images belonging to 45 classes.


In [7]:
# Unfreeze some layers of the base model for fine-tuning
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

In [8]:
# Recompile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator
)

Epoch 1/10


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  5/493 ━━━━━━━━━━━━━━━━━━━━ 34:35 4s/step - accuracy: 0.9470 - loss: 0.1162

KeyboardInterrupt: 

In [ ]:
model.save('model/model_1.h5')

In [4]:
# Evaluate the model
test_loss, test_acc = model.evaluate(valid_generator)
print(f'Test accuracy: {test_acc * 100:.2f}%')

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


106/106 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.9207 - loss: 0.2520
Test accuracy: 91.41%


In [5]:
## Function to predict plant disease for a new image with error message for non-disease images
def predict_plant_disease(img_path, confidence_threshold=0.5):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_class_name = index_to_class[predicted_class]
    confidence = np.max(prediction)

    if confidence < confidence_threshold:
        print(f"Error: The input image is likely not disease. Confidence: {confidence:.2f}")
        # print(predicted_class_name)
    else:
        print(f"Predicted plant disease: {predicted_class_name} with confidence {confidence:.2f}")

    return predicted_class_name

In [6]:

# Example usage with a new image of a plant disease not in the training set
new_image_path = 'dataset/test/Henna/Henna_001.jpg'  # Replace with the actual path to the plant disease
predicted_class_name = predict_plant_disease(new_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted plant disease: Seethapala with confidence 0.69
